J'ai eu l'occasion de travailler récemment [sur les disparités de vaccination dans le Grand-Est en général](https://www.rue89strasbourg.com/age-revenu-esoterisme-pourquoi-le-grand-est-affiche-des-taux-de-vaccination-si-differents-224743), et ses extrêmes alsaciens en particulier. Il a fallu pour cela manipuler différents fichiers fournis par des organisations publiques.

Tous avaient en commun d'être **simples en apparence, mais à manier avec précaution**. Leur préparation en json a requis un panel assez exhaustif d'opérations basiques réalisées avec le [fameux module Python pandas](https://pandas.pydata.org/), parmi lesquelles :

* **la jointure de deux bases** à partir d'une colonne commune
* **le regroupement de valeurs** à partir de valeurs communes dans une même colonne. Par exemple, additionner les données de chaque jour à partir d'une colonne où certains jours apparaissent plusieurs fois
* **la transformation [en tableau croisé dynamique](https://openclassrooms.com/fr/courses/825502-analysez-des-donnees-avec-excel/822596-les-tableaux-croises-dynamiques-1-2)**, en changeant de place les valeurs d'une colonne en colonnes distinctes et en déplaçant les données associées
* **l'application de fonctions** à appliquer sur une colonne ou une base de données entières
* cerise sur le gâteau, j'ai **factorisé autant que possible les manipulations les plus complexes**. Coder a aussi cet intérêt

Bref, une bonne séance d'aérobic statistique avec à l'arrivée trois fichiers interprétables par [la bibliothèque de visualisation D3](https://d3js.org/). 

C'est parti, avec l'import des bibliothèques Python nécessaires :

In [1]:
from math import *
from shapely.geometry import Point
import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

C:\Users\raphi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Données hospitalières et vaccinations

On va d'abord se pencher sur la superposition des données hospitalières (hospitalisations, réanimations, décès à l'hôpital) avec les données de vaccination.

On va librement s'inspirer de [ce travail des journalistes du Financial Times John Burn-Murdoch et David Pilling](https://www.ft.com/content/fa4f248a-a476-491d-a5ce-f128360e9f24) :

![Source : Financial Times](ft_vaccination.jpg)

Plutôt que de placer en miroir les cas détectés de covid et les décès, il s'agit de préparer un json qui va croiser les données citées avant. Commençons par la vaccination départementale. D'abord, lisons le fichier de vaccination au niveau départemental :

In [2]:
df_vacc=pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/f77106ed-6e27-48cf-85b7-daad2b7fce1e", sep=";")
df_vacc.tail()

,dep,jour,n_dose1,n_complet,n_rappel,n_cum_dose1,n_cum_complet,n_cum_rappel,couv_dose1,couv_complet,couv_rappel
35419,95,2021-12-26,191,179,2432,892552,876642,336656,86.9,85.3,32.8
35420,95,2021-12-27,290,362,6016,892842,877004,342672,86.9,85.4,33.4
35421,95,2021-12-28,478,554,10552,893320,877558,353224,86.9,85.4,34.4
35422,95,2021-12-29,517,500,9730,893837,878058,362954,87.0,85.5,35.3
35423,95,2021-12-30,480,481,8450,894317,878539,371404,87.0,85.5,36.1


Colonnes assez compréhensibles, dont on ne va retenir que le département, le jour, et le nombre cumulé de vaccinations complètes :

In [3]:
df_vacc=df_vacc[["dep","jour","n_cum_complet"]]
df_vacc.tail()

,dep,jour,n_cum_complet
35419,95,2021-12-26,876642
35420,95,2021-12-27,877004
35421,95,2021-12-28,877558
35422,95,2021-12-29,878058
35423,95,2021-12-30,878539


A présent, on va garder les départements du Grand-Est, en utilisant la liste de départements associée :

In [4]:
df_vacc=df_vacc[df_vacc["dep"].isin(["08","10","51","52","54","55","57","67","68","88"])].reset_index(drop=True)
df_vacc.tail()

,dep,jour,n_cum_complet
3685,88,2021-12-26,279343
3686,88,2021-12-27,279511
3687,88,2021-12-28,279654
3688,88,2021-12-29,279760
3689,88,2021-12-30,279849


On va à présent regrouper ces chiffres départementaux par journée, en les additionnant. Une seule ligne, et l'affaire est réglée :

In [5]:
df_vacc=df_vacc.groupby(df_vacc["jour"]).sum().reset_index()
df_vacc.tail()

,jour,n_cum_complet
364,2021-12-26,4135774
365,2021-12-27,4138035
366,2021-12-28,4140390
367,2021-12-29,4142676
368,2021-12-30,4144447


*You know what ?* C'est parfaitement conforme aux chiffres régionaux [trouvables sur le portail Geodes](https://geodes.santepubliquefrance.fr/#c=indicator&i=vacsi12.n_cum_complet&s=2021-12-30&t=a01&view=map1) ! On peut dès lors passer [aux données hospitalières fournies par Santé Publique France](https://www.data.gouv.fr/fr/datasets/donnees-hospitalieres-relatives-a-lepidemie-de-covid-19/) :

In [6]:
df_hosp=pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/6fadff46-9efd-4c53-942a-54aca783c30c", sep=";")
df_hosp.tail()

,dep,jour,incid_hosp,incid_rea,incid_dc,incid_rad
66805,978,2021-12-29,0,0,0,0
66806,978,2021-12-30,0,0,0,0
66807,978,2021-12-31,0,0,0,0
66808,978,2022-01-01,0,0,0,0
66809,978,2022-01-02,0,0,0,0


On repart sur le même principe, d'abord une sélection départementale :

In [7]:
df_hosp=df_hosp[df_hosp["dep"].isin(["08","10","51","52","54","55","57","67","68","88"])].reset_index(drop=True)
df_hosp["dep"].unique()

array(['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],
      dtype=object)

Ensuite, un regrroupement en additionnant les données de chaque jour :

In [8]:
df_hosp=df_hosp.groupby(df_hosp["jour"]).sum()
df_hosp=df_hosp.reset_index()
df_hosp.tail()

,jour,incid_hosp,incid_rea,incid_dc,incid_rad
650,2021-12-29,115,25,15,70
651,2021-12-30,131,25,11,102
652,2021-12-31,121,25,19,99
653,2022-01-01,93,20,19,54
654,2022-01-02,52,4,7,15


On va à présent ne retenir que les colonnes qui nous intéressent, et ensuite fusionner la DataFrame des vaccinations avec celle des hospitalisations.

Pour cela, on va prendre en référence **la colonne commune jour**, et fusionner afin de garder toutes les dates présentes sur la DataFrame d'hospitalisation. Les dates absentes de l'autre DF vont être colmatées par des NaN, que l'on va remplacer sans hésiter par des 0, grâce à la [fonction fillna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) :

In [9]:
df_hosp=df_hosp[["jour", "incid_hosp", "incid_rea", "incid_dc"]]
df_vacc_hosp=df_vacc.merge(df_hosp, on="jour", how='right')
df_vacc_hosp=df_vacc_hosp.fillna(0)
df_vacc_hosp["n_cum_complet"]=df_vacc_hosp["n_cum_complet"].astype(int)
df_vacc_hosp.head()

,jour,n_cum_complet,incid_hosp,incid_rea,incid_dc
0,2020-03-19,0,438,69,38
1,2020-03-20,0,263,45,27
2,2020-03-21,0,313,65,44
3,2020-03-22,0,291,63,47
4,2020-03-23,0,416,81,63


Les premières dates ont bien été remplacées par des 0 côté vaccination, et cela vaut aussi pour la fin de DF :

In [10]:
df_vacc_hosp.tail()

,jour,n_cum_complet,incid_hosp,incid_rea,incid_dc
650,2021-12-29,4142676,115,25,15
651,2021-12-30,4144447,131,25,11
652,2021-12-31,0,121,25,19
653,2022-01-01,0,93,20,19
654,2022-01-02,0,52,4,7


On n'a pas d'autres choix que de supprimer les trois dernières lignes, comme ceci :

In [11]:
df_vacc_hosp=df_vacc_hosp.iloc[:-3]
# la notation ci-dessus sert à redécouper une liste sur le modèle [start:stop:step]
# [:3] pourrait s'écrire [0:3:1] ; on part de l'indice zéro, on s'arrête à -3 (3e valeur en partant de la fin), et un pas de 1 à chaque fois
df_vacc_hosp.tail()

,jour,n_cum_complet,incid_hosp,incid_rea,incid_dc
647,2021-12-26,4135774,37,7,9
648,2021-12-27,4138035,107,24,21
649,2021-12-28,4140390,151,34,14
650,2021-12-29,4142676,115,25,15
651,2021-12-30,4144447,131,25,11


Tout est d'équerre, on peut à présent renommer nos colonnes :

In [12]:
df_vacc_hosp=df_vacc_hosp.rename(columns={"n_cum_complet":"vac_cum","incid_hosp":"hosp","incid_rea":"rea","incid_dc":"dc"})
df_vacc_hosp.tail()

,jour,vac_cum,hosp,rea,dc
647,2021-12-26,4135774,37,7,9
648,2021-12-27,4138035,107,24,21
649,2021-12-28,4140390,151,34,14
650,2021-12-29,4142676,115,25,15
651,2021-12-30,4144447,131,25,11


Enfin, on va dresser pour chaque donnée hospitalière **une moyenne glissante sur sept jours** que l'on superposera pour plus de lisibilité :

In [13]:
for d in ["hosp","rea","dc"]:
    df_vacc_hosp["mg_"+d]=df_vacc_hosp[d].rolling(7, min_periods=1, center = True).mean()
df_vacc_hosp.tail()

,jour,vac_cum,hosp,rea,dc,mg_hosp,mg_rea,mg_dc
647,2021-12-26,4135774,37,7,9,96.428571,23.285714,15.142857
648,2021-12-27,4138035,107,24,21,98.571429,22.571429,13.571429
649,2021-12-28,4140390,151,34,14,102.000000,22.666667,14.000000
650,2021-12-29,4142676,115,25,15,108.200000,23.000000,14.000000
651,2021-12-30,4144447,131,25,11,126.000000,27.000000,15.250000


Plus qu'à sauvegarder en json, et réserver le fichier :

In [14]:
df_vacc_hosp.to_json("data/vac_hosp.json",orient="records")

# Cartographie par EPCI

Attaquons le gros morceau de ce tutoriel, avec **une cartographie de la vaccination par Etablissement Public de Coopération Intercommunale** (EPCI). L'objectif est d'arriver à ce type de cartographie, avec navigation par semaine :

![Source : L'Assurance maladie](carto_ecpi.jpg)

On est dans le cas d'école du jeu de données **simple en apparence**, mais à manier avec précaution. Ce dernier est fourni par la Caisse nationale de l'assurance maladie [sur le portail Datagouv](https://www.data.gouv.fr/fr/datasets/donnees-vaccination-par-epci/). On va entamer les festivités en téléchargeant la version la plus récente du fichier. 

(NB : ce téléchargement peut prendre du temps en raison de la taille du fichier)

In [2]:
vacepci=pd.read_csv("https://datavaccin-covid.ameli.fr/explore/dataset/donnees-de-vaccination-par-epci/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false",sep=";", parse_dates=["date_reference","date"],low_memory=False)
vacepci.tail()

,date_reference,semaine_injection,epci,libelle_epci,population_carto,classe_age,libelle_classe_age,effectif_1_inj,effectif_termine,effectif_cumu_1_inj,effectif_cumu_termine,taux_1_inj,taux_termine,taux_cumu_1_inj,taux_cumu_termine,date,reg_code
465965,2021-12-26,2021-41,999999999,NaN,1166170,TOUT_AGE,Tout âge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-17,NaN
465966,2021-12-26,2021-43,999999999,NaN,1166170,TOUT_AGE,Tout âge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-31,NaN
465967,2021-12-26,2021-44,999999999,NaN,1166170,TOUT_AGE,Tout âge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-07,NaN
465968,2021-12-26,2021-46,999999999,NaN,1166170,TOUT_AGE,Tout âge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-21,NaN
465969,2021-12-26,2021-50,999999999,NaN,1166170,TOUT_AGE,Tout âge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-19,NaN


On va recopier ici quelques rappels utiles sur les données stipulés sur Datagouv :

* la population de référence pour le calcul des effectifs de patients vaccinés et pour le calcul des taux est **la population de la cartographie des pathologies et des dépenses de l’Assurance Maladie**. On peut donc constater des différences avec les effectifs exportés précédemment, le plus important est surtout que les ordres de grandeur soient comparables *(nb : ce qui est le cas, en prenant en compte le point suivant)*
* seuls les patients vaccinés faisant partie de cette population sont pris en compte dans les effectifs **(> 95% des vaccinés)**
* les patients **dont le département de résidence ou la commune est inconnu(e) sont rattachés à l’EPCI 999999999**, qu'on voit apparaître fort logiquement en toute fin de la DataFrame vacepci

A ce stade, nous pouvons commencer à faire quelques sélections, avec :

* une retenue des EPCI localisées dans le Grand-Est (code région 44). On va chercher dans la colonne les chaînes de caractères qui incluent ce code, pour ne pas oublier la Communauté de communes des Savoir-Faire (à cheval sur la Bourgogne-Franche Comté)
* retenir les semaines d'injection marquées en 2021
* considérer toutes les tranches d'âge

On va également changer les codes d'ecpi en nombres entiers et ranger la DataFrame écrémée dans l'ordre croissant de date et code d'ECPI. Précaution importante : on va aussi copier immédiatement la DF vacepci car elle met beaucoup de temps à se charger et qu'il serait dommage d'avoir à la réimporter.

Bref, c'est parti :

In [3]:
vacepci_ge=vacepci.copy()
vacepci_ge=vacepci_ge[vacepci_ge["reg_code"].str.contains("44", na=False)]
vacepci_ge=vacepci_ge[vacepci_ge["semaine_injection"].str.contains("2021", na=False)]
vacepci_ge=vacepci_ge[vacepci_ge["classe_age"]=="TOUT_AGE"]
vacepci_ge["epci"]=vacepci_ge["epci"].astype(int)
vacepci_ge.sort_values(["date","epci"],ascending=[True,True],inplace=True)
vacepci_ge.reset_index(inplace=True, drop=True)
vacepci_ge=vacepci_ge[["date_reference","date","semaine_injection","epci","libelle_epci","population_carto","effectif_cumu_termine","taux_cumu_termine"]]
vacepci_ge

,date_reference,date,semaine_injection,epci,libelle_epci,population_carto,effectif_cumu_termine,taux_cumu_termine
0,2021-12-26,2021-01-10,2021-01,200000545,CC DES PORTES DE ROMILLY SUR SEINE,18070,NaN,NaN
1,2021-12-26,2021-01-10,2021-01,200005957,CC DE LA REGION DE RAMBERVILLERS,12450,NaN,NaN
2,2021-12-26,2021-01-10,2021-01,200006716,CC DU NOGENTAIS,15390,NaN,NaN
3,2021-12-26,2021-01-10,2021-01,200013050,CC SAUER PECHELBRONN,16890,NaN,NaN
4,2021-12-26,2021-01-10,2021-01,200027308,CC D AUBERIVE VINGEANNE ET MONTSAUGEONNAIS,7910,NaN,NaN
...,...,...,...,...,...,...,...,...
7594,2021-12-26,2021-12-26,2021-51,246800569,CC DE LA REGION DE GUEBWILLER,36980,26530.0,0.717
7595,2021-12-26,2021-12-26,2021-51,246800577,CC DU PAYS DE RIBEAUVILLE,17610,13360.0,0.759
7596,2021-12-26,2021-12-26,2021-51,246800585,CC DE LA VALLEE DE MUNSTER,15800,11360.0,0.719
7597,2021-12-26,2021-12-26,2021-51,246800676,CC DE LA VALLEE DE LA DOLLER ET DU SOULTZBACH,15680,11310.0,0.721


**Beaucoup de NaN dans les premières valeurs**, on va regarder les cinq premières lignes d'une même communauté de communes, par exemple celle de Romilly-sur-Seine :

In [4]:
vacepci_ge[vacepci_ge["epci"]==200000545].iloc[:5]

,date_reference,date,semaine_injection,epci,libelle_epci,population_carto,effectif_cumu_termine,taux_cumu_termine
0,2021-12-26,2021-01-10,2021-01,200000545,CC DES PORTES DE ROMILLY SUR SEINE,18070,NaN,NaN
149,2021-12-26,2021-01-17,2021-02,200000545,CC DES PORTES DE ROMILLY SUR SEINE,18070,NaN,NaN
298,2021-12-26,2021-01-24,2021-03,200000545,CC DES PORTES DE ROMILLY SUR SEINE,18070,NaN,0.000
447,2021-12-26,2021-01-31,2021-04,200000545,CC DES PORTES DE ROMILLY SUR SEINE,18070,20.0,0.001
596,2021-12-26,2021-02-07,2021-05,200000545,CC DES PORTES DE ROMILLY SUR SEINE,18070,70.0,0.004


On a des données complètes à partir de la semaine 4 de 2021, mais combien de lignes ont des valeurs non renseignées ? Réponse :

In [5]:
len(vacepci_ge[vacepci_ge["effectif_cumu_termine"].isna()])

499

On n'insistera jamais assez sur la facilité avec laquelle on peut interroger une BDD avec pandas. **Le gain de temps est considérable** en comparaison du maniement parfois hasardeux d'un logiciel tableur, et **on peut en plus recycler/adapter des scripts qui ont déjà fait leurs preuves**.

Par exemple, pour afficher les semaines d'injection pour lesquelles on a des NaN :

In [6]:
vacepci_ge[vacepci_ge["effectif_cumu_termine"].isna()]["semaine_injection"].unique()

array(['2021-01', '2021-02', '2021-03', '2021-04', '2021-05'],
      dtype=object)

Ou regarder les dates concernées (en réécrivant à peine la commande précédente) :

In [7]:
vacepci_ge[vacepci_ge["effectif_cumu_termine"].isna()]["date"].unique()

array(['2021-01-10T00:00:00.000000000', '2021-01-17T00:00:00.000000000',
       '2021-01-24T00:00:00.000000000', '2021-01-31T00:00:00.000000000',
       '2021-02-07T00:00:00.000000000'], dtype='datetime64[ns]')

Traduction : aucune valeur NaN au-delà de la date du 7 février. On va donc ne retenir dans la DataFrame que les jours postérieurs à cette date :

In [8]:
vacepci_ge=vacepci_ge[vacepci_ge["date"]>"2021-02-07T00:00:00.000000000'"]
len(vacepci_ge[vacepci_ge["effectif_cumu_termine"].isna()])

0

Et maintenant ? On va recalculer le taux avec une multiplication par 100, ainsi que convertir les effectifs cumulés en nombres entiers :

In [9]:
vacepci_ge["taux_cumu_termine"]=vacepci_ge["taux_cumu_termine"]*100
vacepci_ge["effectif_cumu_termine"]=vacepci_ge["effectif_cumu_termine"].astype("int")
vacepci_ge.tail()

C:\Users\raphi\AppData\Local\Temp/ipykernel_37252/4207991193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacepci_ge["taux_cumu_termine"]=vacepci_ge["taux_cumu_termine"]*100
C:\Users\raphi\AppData\Local\Temp/ipykernel_37252/4207991193.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacepci_ge["effectif_cumu_termine"]=vacepci_ge["effectif_cumu_termine"].astype("int")


,date_reference,date,semaine_injection,epci,libelle_epci,population_carto,effectif_cumu_termine,taux_cumu_termine
7594,2021-12-26,2021-12-26,2021-51,246800569,CC DE LA REGION DE GUEBWILLER,36980,26530,71.7
7595,2021-12-26,2021-12-26,2021-51,246800577,CC DU PAYS DE RIBEAUVILLE,17610,13360,75.9
7596,2021-12-26,2021-12-26,2021-51,246800585,CC DE LA VALLEE DE MUNSTER,15800,11360,71.9
7597,2021-12-26,2021-12-26,2021-51,246800676,CC DE LA VALLEE DE LA DOLLER ET DU SOULTZBACH,15680,11310,72.1
7598,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,108540,77590,71.5


On va vérifier que rien ne dépasse dans notre DataFrame. Pour les opérations que l'on s'apprête à faire, **il est indispensable que les EPCI soient déclarés dans le même ordre**, et dans le même nombre. On va déjà vérifier qu'aucune ligne ne manque comme ceci :

In [10]:
print(len(vacepci_ge["epci"].unique()),"EPCI uniques dans le jeu de données")
print(len(vacepci_ge["semaine_injection"].unique()),"semaines uniques")
print(len(vacepci_ge),"lignes totales")
print(len(vacepci_ge["epci"].unique())*len(vacepci_ge["semaine_injection"].unique()),"lignes attendues")

149 EPCI uniques dans le jeu de données
46 semaines uniques
6854 lignes totales
6854 lignes attendues


Ca sent plutôt bon, mais on va encore vérifier semaine par semaine que rien ne dépasse :

In [11]:
for s in vacepci_ge["semaine_injection"].unique():
    DF_trans=vacepci_ge[vacepci_ge["semaine_injection"]==s]
    if len(DF_trans["epci"].unique())!=len(vacepci_ge["epci"].unique()):
        print("Oups, il manque des ECPI pour la semaine",s)

Ces précautions prises, on va passer à la partie géographique. Les données sont tirées [de la Base nationale sur l'intercommunalité](https://www.banatic.interieur.gouv.fr/V5/cartographie/cartographie.php) (BANATIC), dont j'ai sauvegardé une copie en local.

Elle n'apparaît pas dans ce répertoire Github, donc vous devrez juste changer le chemin de fichier  :

In [12]:
epci = gpd.read_file(r"C:\Users\raphi\Documents\Ressources\France\EPCI\2021\Métropole")
epci.crs

<Derived Projected CRS: EPSG:2154>
Name: RGF93 v1 / Lambert-93
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: France - onshore and offshore, mainland and Corsica.
- bounds: (-9.86, 41.15, 10.38, 51.56)
Coordinate Operation:
- name: Lambert-93
- method: Lambert Conic Conformal (2SP)
Datum: Reseau Geodesique Francais 1993 v1
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

La projection utilisée [est la Lambert-93](https://geodesie.ign.fr/contenu/fichiers/PlaqLambert93.pdf), la franco-français qui garde bien de chez nous, et on va très vite reformater tout ça en Mercator :

In [13]:
epci['geometry']=epci['geometry'].to_crs(epsg=4326)
epci.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

A présent, on va refaire une sélection des EPCI du Grand-Est, grâce au code régional :

In [14]:
epci_ge=epci[epci["CODE_REG"]=="44"]
epci_ge.reset_index(inplace=True, drop=True)
epci_ge.tail()

,ID_GEOFLA,STATUT,X_CHF_LIEU,Y_CHF_LIEU,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,CODE_DEPT,NOM_DEPT,...,CODE_ARR,DEPT,SIREN,RAISON_SOC,NJ,DEP_COM,INSEE,SIREN_MEMB,NOM_MEMBRE,geometry
144,1094,Commune simple,1014193,6765308,1013039,6765429,480,18989,68,HAUT-RHIN,...,6,68,246800569.0,CC de la Région de Guebwiller,CCFPU,68,68030,216800300.0,Bergholtzzell,"MULTIPOLYGON (((7.32988 47.91798, 7.32848 47.9..."
145,1095,Commune simple,1022161,6796364,1021653,6796819,363,16651,68,HAUT-RHIN,...,2,68,246800577.0,CC du Pays de Ribeauvillé,CCFPU,68,68023,216800235.0,Beblenheim,"POLYGON ((7.47694 48.20353, 7.47610 48.20249, ..."
146,1096,Commune simple,1007342,6778229,1006887,6778032,694,19562,68,HAUT-RHIN,...,2,68,246800585.0,CC de la Vallée de Munster,CCFPU,68,68083,216800839.0,Eschbach-au-Val,"POLYGON ((7.23590 48.02608, 7.23582 48.02599, ..."
147,1097,Commune simple,1000527,6748892,1000120,6749169,540,16062,68,HAUT-RHIN,...,6,68,246800676.0,CC de la Vallée de la Doller et du Soultzbach,CCFPU,68,68060,216800607.0,Burnhaupt-le-Haut,"POLYGON ((7.18993 47.72840, 7.18980 47.72764, ..."
148,1098,Commune simple,1024829,6784513,1024681,6784748,250,24426,68,HAUT-RHIN,...,2,68,246800726.0,CA Colmar Agglomération,CA,68,68038,216800383.0,Bischwihr,"POLYGON ((7.51859 48.12776, 7.51819 48.12774, ..."


On peut dès lors ne sélectionner que les colonnes intéressantes, et en renommer deux sur le même modèle que la DF de données :

In [15]:
epci_ge=epci_ge[["SIREN", "RAISON_SOC", "geometry"]]
epci_ge["SIREN"]=epci_ge["SIREN"].astype(int)
epci_ge=epci_ge.rename(columns={"SIREN":"epci", "RAISON_SOC":"libelle_epci"})
epci_ge

,epci,libelle_epci,geometry
0,200000545,CC des Portes de Romilly sur Seine,"POLYGON ((3.82590 48.51517, 3.82497 48.51338, ..."
1,200005957,CC de la Région de Rambervillers,"POLYGON ((6.81509 48.39498, 6.81477 48.39440, ..."
2,200006716,CC du Nogentais,"POLYGON ((3.64363 48.53626, 3.64299 48.53564, ..."
3,200013050,CC Sauer-Pechelbronn,"POLYGON ((7.86687 48.93731, 7.86561 48.93457, ..."
4,200027308,CC d'Auberive Vingeanne et Montsaugeonnais,"POLYGON ((5.48453 47.68626, 5.48452 47.68612, ..."
...,...,...,...
144,246800569,CC de la Région de Guebwiller,"MULTIPOLYGON (((7.32988 47.91798, 7.32848 47.9..."
145,246800577,CC du Pays de Ribeauvillé,"POLYGON ((7.47694 48.20353, 7.47610 48.20249, ..."
146,246800585,CC de la Vallée de Munster,"POLYGON ((7.23590 48.02608, 7.23582 48.02599, ..."
147,246800676,CC de la Vallée de la Doller et du Soultzbach,"POLYGON ((7.18993 47.72840, 7.18980 47.72764, ..."


Pour la suite, on va réserver une nouvelle DF qui contient les 149 premiers EPCI renseignés dans vacepci_ge :

In [16]:
tampon=vacepci_ge.copy()[["epci"]].iloc[:len(epci_ge["epci"].unique())]
tampon.tail()

,epci
889,246800569
890,246800577
891,246800585
892,246800676
893,246800726


Vérifions encore une fois que rien ne dépasse pour les EPCI des deux DataFrames :

In [17]:
for r,c in zip(epci_ge["epci"],tampon["epci"].unique()):
    print (r,c)

200000545 200000545
200005957 200005957
200006716 200006716
200013050 200013050
200027308 200027308
200030526 200030526
200033025 200033025
200033868 200033868
200034270 200034270
200034635 200034635
200034718 200034718
200034874 200034874
200035772 200035772
200036465 200036465
200039865 200039865
200039907 200039907
200039949 200039949
200040137 200040137
200040178 200040178
200041283 200041283
200041325 200041325
200041515 200041515
200041622 200041622
200041630 200041630
200042000 200042000
200042620 200042620
200042703 200042703
200042992 200042992
200043156 200043156
200043438 200043438
200043693 200043693
200044253 200044253
200049187 200049187
200066009 200066009
200066025 200066025
200066033 200066033
200066041 200066041
200066058 200066058
200066108 200066108
200066116 200066116
200066132 200066132
200066140 200066140
200066157 200066157
200066165 200066165
200066173 200066173
200066835 200066835
200066850 200066850
200066876 200066876
200066892 200066892
200067213 200067213


Superbe, on va maintenant définir **une fonction qui va rassembler dans un dico Python différentes données** récoltées dans les colonnes distinctes d'une DataFrame :

In [18]:
def dico_secol(DF:pd.core.frame.DataFrame, col_ref:str, col_cible:str, col_val:str, col_cont:str):
    """
        Cette fonction rassemble dans une liste de dictionnaire deux types de valeurs (absolue et relative)
        groupée par valeur d'une colonne cible.
        Par exemple : rassembler les effectifs cumulés et taux de vaccination complète par date à partir d'une DF.
    """
    l_finale=[]
    for c in DF[col_ref].unique():
        DF_trans=DF[DF[col_ref]==c]
        l_finale.append({r:{col_val:v,col_cont:c} for r,v,c in zip(DF_trans[col_cible],DF_trans[col_val],DF_trans[col_cont])})
    return l_finale

Ce qui donne, pour la première valeur de la DF, cette liste de dicos liée au premier EPCI de notre sélection :

In [19]:
dico_secol(vacepci_ge,"epci","semaine_injection","taux_cumu_termine","effectif_cumu_termine")[0]

{'2021-06': {'taux_cumu_termine': 0.8, 'effectif_cumu_termine': 150},
 '2021-07': {'taux_cumu_termine': 1.5, 'effectif_cumu_termine': 270},
 '2021-08': {'taux_cumu_termine': 1.9, 'effectif_cumu_termine': 350},
 '2021-09': {'taux_cumu_termine': 2.7, 'effectif_cumu_termine': 490},
 '2021-10': {'taux_cumu_termine': 3.1, 'effectif_cumu_termine': 560},
 '2021-11': {'taux_cumu_termine': 3.5000000000000004,
  'effectif_cumu_termine': 620},
 '2021-12': {'taux_cumu_termine': 3.8, 'effectif_cumu_termine': 690},
 '2021-13': {'taux_cumu_termine': 5.3, 'effectif_cumu_termine': 950},
 '2021-14': {'taux_cumu_termine': 6.1, 'effectif_cumu_termine': 1110},
 '2021-15': {'taux_cumu_termine': 8.1, 'effectif_cumu_termine': 1470},
 '2021-16': {'taux_cumu_termine': 10.299999999999999,
  'effectif_cumu_termine': 1860},
 '2021-17': {'taux_cumu_termine': 11.5, 'effectif_cumu_termine': 2070},
 '2021-18': {'taux_cumu_termine': 12.8, 'effectif_cumu_termine': 2320},
 '2021-19': {'taux_cumu_termine': 14.899999999999

Comme les choses sont assez bien bétonnées, on a : 
* chaque liste de dictionnaires qui **correspond à un EPCI distinct**
* l'ordre des établissements **suit exactement le même que la DF tampon**

C'est donc parti pour ajouter une nouvelle colonne à cette dernière :

In [20]:
tampon["histo_vac"]=dico_secol(vacepci_ge,"epci","semaine_injection","taux_cumu_termine","effectif_cumu_termine")
tampon.tail()

,epci,histo_vac
889,246800569,"{'2021-06': {'taux_cumu_termine': 0.3, 'effect..."
890,246800577,"{'2021-06': {'taux_cumu_termine': 0.4, 'effect..."
891,246800585,"{'2021-06': {'taux_cumu_termine': 0.5, 'effect..."
892,246800676,"{'2021-06': {'taux_cumu_termine': 0.3, 'effect..."
893,246800726,"{'2021-06': {'taux_cumu_termine': 1.0, 'effect..."


On va à présent fusionner tampon avec la DataFrame géographique :

In [21]:
geo_epci = tampon.merge(epci_ge, on="epci")
geo_epci.tail()

,epci,histo_vac,libelle_epci,geometry
144,246800569,"{'2021-06': {'taux_cumu_termine': 0.3, 'effect...",CC de la Région de Guebwiller,"MULTIPOLYGON (((7.32988 47.91798, 7.32848 47.9..."
145,246800577,"{'2021-06': {'taux_cumu_termine': 0.4, 'effect...",CC du Pays de Ribeauvillé,"POLYGON ((7.47694 48.20353, 7.47610 48.20249, ..."
146,246800585,"{'2021-06': {'taux_cumu_termine': 0.5, 'effect...",CC de la Vallée de Munster,"POLYGON ((7.23590 48.02608, 7.23582 48.02599, ..."
147,246800676,"{'2021-06': {'taux_cumu_termine': 0.3, 'effect...",CC de la Vallée de la Doller et du Soultzbach,"POLYGON ((7.18993 47.72840, 7.18980 47.72764, ..."
148,246800726,"{'2021-06': {'taux_cumu_termine': 1.0, 'effect...",CA Colmar Agglomération,"POLYGON ((7.51859 48.12776, 7.51819 48.12774, ..."


Puis on retransforme la DataFrame en GeoDataFrame, **avec une simplification géométrique** afin d'éviter un fichier est lourd :

In [22]:
geo_epci = gpd.GeoDataFrame(geo_epci, geometry=geo_epci["geometry"])
geo_epci["geometry"]=geo_epci["geometry"].simplify(.0035)

Et plus qu'à enregistrer le fichier json :

In [23]:
geo_epci.to_file("data/vac_epci.json", driver="GeoJSON")

## Ajout des départements et villes repères

Pour les autres calques de la carte, rien que du très classique, avec quelques manips pour récupérer les départements d'Alsace.

Premier temps, on récupère le fichier d'OpenStreetMap et on vire les colonnes inutiles :

In [24]:
deps = gpd.read_file("http://osm13.openstreetmap.fr/~cquest/openfla/export/departements-20180101-shp.zip")
deps.drop(columns=["wikipedia","surf_km2","nuts3"], inplace=True)
deps.head()

,code_insee,nom,geometry
0,974,La Réunion,"MULTIPOLYGON (((55.21643 -21.03904, 55.21652 -..."
1,11,Aude,"POLYGON ((1.68872 43.27368, 1.69001 43.27423, ..."
2,43,Haute-Loire,"POLYGON ((3.08206 45.28988, 3.08209 45.29031, ..."
3,13,Bouches-du-Rhône,"MULTIPOLYGON (((4.23014 43.46047, 4.23025 43.4..."
4,47,Lot-et-Garonne,"POLYGON ((-0.14058 44.22648, -0.12931 44.23218..."


Ensuite, on récupère les départements du Grand-Est :

In [25]:
dep_ge=deps[deps["code_insee"].isin(["08","10","51","52","54","55","57","67","68","88"])].reset_index()
dep_ge.drop(columns="index",inplace=True)
dep_ge

,code_insee,nom,geometry
0,54,Meurthe-et-Moselle,"MULTIPOLYGON (((5.44726 49.40393, 5.44732 49.4..."
1,57,Moselle,"POLYGON ((5.89185 49.49570, 5.89189 49.49582, ..."
2,67,Bas-Rhin,"POLYGON ((6.94063 48.92412, 6.94105 48.92466, ..."
3,51,Marne,"POLYGON ((3.39589 48.75908, 3.39598 48.75932, ..."
4,08,Ardennes,"POLYGON ((4.02460 49.62126, 4.02597 49.62179, ..."
5,68,Haut-Rhin,"POLYGON ((6.84132 47.81936, 6.84293 47.82038, ..."
6,55,Meuse,"POLYGON ((4.88846 48.80060, 4.88854 48.80062, ..."
7,10,Aube,"POLYGON ((3.38364 48.47958, 3.38370 48.47963, ..."
8,52,Haute-Marne,"POLYGON ((4.62751 48.46717, 4.62755 48.46725, ..."
9,88,Vosges,"POLYGON ((5.39361 48.39177, 5.39373 48.39181, ..."


Et enfin, on ne garde que les contours et on enregistre le résultat en json :

In [26]:
dep_ge["geometry"]=dep_ge["geometry"].simplify(.0035)
contours = dep_ge.boundary
contours.to_file("data/contours.json", driver="GeoJSON")

Le plan est exactement le même pour les villes repères. Récupération d'un fichier OSM...

In [27]:
communes = gpd.read_file("https://osm13.openstreetmap.fr/~cquest/openfla/export/communes-20210101-shp.zip")
communes.drop(columns=["wikipedia","surf_ha"], inplace=True)
communes.head()

,insee,nom,geometry
0,30225,Sabran,"POLYGON ((4.49703 44.14344, 4.49727 44.14398, ..."
1,26334,Salettes,"POLYGON ((4.94213 44.57481, 4.94283 44.57501, ..."
2,04200,Salignac,"POLYGON ((5.95337 44.15613, 5.95341 44.15649, ..."
3,04179,Saint-Geniez,"POLYGON ((5.98593 44.23280, 5.98654 44.23292, ..."
4,05162,La Saulce,"POLYGON ((5.98920 44.41892, 5.98921 44.41894, ..."


...sélection à partir d'une liste de codes INSEE... :

In [28]:
reperes=communes[communes["insee"].isin(["08105","57463","52121","54395","88160","55545","51454","68224","67482","51108","10387","68066"])].reset_index(drop=True)
reperes

,insee,nom,geometry
0,52121,Chaumont,"POLYGON ((5.08324 48.07983, 5.08343 48.08025, ..."
1,67482,Strasbourg,"POLYGON ((7.68814 48.59947, 7.68970 48.59983, ..."
2,10387,Troyes,"POLYGON ((4.04117 48.30688, 4.04117 48.30694, ..."
3,51108,Châlons-en-Champagne,"POLYGON ((4.32957 48.96528, 4.32965 48.96533, ..."
4,54395,Nancy,"POLYGON ((6.13429 48.69203, 6.13831 48.69221, ..."
5,57463,Metz,"POLYGON ((6.13600 49.13788, 6.13739 49.13837, ..."
6,88160,Épinal,"POLYGON ((6.39376 48.12614, 6.39386 48.12706, ..."
7,68066,Colmar,"POLYGON ((7.31548 48.07017, 7.31567 48.07098, ..."
8,55545,Verdun,"POLYGON ((5.28097 49.13623, 5.28098 49.13627, ..."
9,51454,Reims,"POLYGON ((3.98582 49.29335, 3.98635 49.29353, ..."


...transformation des polygones en points... :

In [29]:
reperes.drop(columns="insee",inplace=True)
reperes["geometry"]=reperes["geometry"].centroid
reperes

C:\Users\raphi\AppData\Local\Temp/ipykernel_37252/1936713536.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reperes["geometry"]=reperes["geometry"].centroid


,nom,geometry
0,Chaumont,POINT (5.13976 48.09791)
1,Strasbourg,POINT (7.76776 48.57125)
2,Troyes,POINT (4.07844 48.29675)
3,Châlons-en-Champagne,POINT (4.37851 48.96441)
4,Nancy,POINT (6.17528 48.69008)
5,Metz,POINT (6.19601 49.10821)
6,Épinal,POINT (6.47987 48.16323)
7,Colmar,POINT (7.38489 48.10995)
8,Verdun,POINT (5.36278 49.14565)
9,Reims,POINT (4.04304 49.25076)


... et enregistrement du fichier final :

In [30]:
reperes.to_file("data/villes.json", driver="GeoJSON")

# Regroupement régional et minimum/maximum

Après visualisation de la carte, il s'avère intéressant de **comparer le Grand-Est avec ses deux extrêmes alsaciens pour la semaine 49** (la communauté de communes du Kochersberg et l'agglomération de Mulhouse).

On va encore une fois y aller étape par étape, en copiant la DF chargée :

In [31]:
compavac=vacepci.copy()
compavac["epci"]=compavac["epci"].astype(int)
compavac=compavac[compavac["reg_code"].str.contains("44", na=False)]
compavac=compavac[compavac["date"]>"2021-02-07T00:00:00.000000000'"]
compavac.sort_values(["date","epci"],ascending=[True,True],inplace=True)
compavac.reset_index(inplace=True, drop=True)
compavac=compavac[["date_reference","date","semaine_injection","epci","libelle_epci", "libelle_classe_age","population_carto","effectif_cumu_termine","taux_cumu_termine"]]
compavac.tail()

,date_reference,date,semaine_injection,epci,libelle_epci,libelle_classe_age,population_carto,effectif_cumu_termine,taux_cumu_termine
47973,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,Tout âge,108540,77590.0,0.715
47974,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,moins de 20 ans,21520,6160.0,0.286
47975,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,de 40 à 54 ans,21590,17780.0,0.824
47976,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,de 65 à 74 ans,12850,11280.0,0.878
47977,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,75 ans et plus,11960,10260.0,0.858


Cette fois-ci, on va purger les lignes associés à l'ensemble des classes d'âge. Ce qui nous intéresse, ce sont les classes d'âge distinctes, pas le total de toutes ces tranches :

In [32]:
compavac=compavac[compavac["libelle_classe_age"]!="Tout âge"]
compavac.reset_index(inplace=True, drop=True)
compavac.tail()

,date_reference,date,semaine_injection,epci,libelle_epci,libelle_classe_age,population_carto,effectif_cumu_termine,taux_cumu_termine
41119,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,de 20 à 39 ans,25770,19260.0,0.747
41120,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,moins de 20 ans,21520,6160.0,0.286
41121,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,de 40 à 54 ans,21590,17780.0,0.824
41122,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,de 65 à 74 ans,12850,11280.0,0.878
41123,2021-12-26,2021-12-26,2021-51,246800726,CA COLMAR AGGLOMERATION,75 ans et plus,11960,10260.0,0.858


Ces classees d'âge peuvent être affichées comme ceci :

In [33]:
compavac["libelle_classe_age"].unique()

array(['moins de 20 ans', 'de 40 à 54 ans', 'de 20 à 39 ans',
       'de 65 à 74 ans', '75 ans et plus', 'de 55 à 64 ans'], dtype=object)

On va anticiper et regarder combien de lignes de chaque classe d'âge sont concernées par des effectifs non renseignés (NaN). Comme d'habitude, cela se règle en une ligne :

In [34]:
compavac[compavac["effectif_cumu_termine"].isna()]["libelle_classe_age"].value_counts()

moins de 20 ans    2365
de 20 à 39 ans      171
de 65 à 74 ans       84
de 40 à 54 ans        9
de 55 à 64 ans        9
75 ans et plus        6
Name: libelle_classe_age, dtype: int64

Il y a **une surreprésentation des moins de 20 ans**, la catégorie qui a eu le plus tard accès complet à la vaccination. On peut prolonger cette observation avec le taux de vaccinations complètes :

In [35]:
compavac[compavac["taux_cumu_termine"].isna()]["libelle_classe_age"].value_counts()

moins de 20 ans    1064
de 65 à 74 ans        1
Name: libelle_classe_age, dtype: int64

Presque exclusivement des moins de 20 ans... Complétons ce panorama en affichant les semaines pour lesquelles on a :
* des effectifs en NaN
* une classe d'âge correspondant aux moins de 20 ans

In [36]:
compavac[(compavac["effectif_cumu_termine"].isna())&(compavac["libelle_classe_age"]=="moins de 20 ans")]["semaine_injection"].unique()

array(['2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11',
       '2021-12', '2021-13', '2021-14', '2021-15', '2021-16', '2021-17',
       '2021-18', '2021-19', '2021-20', '2021-21', '2021-22', '2021-23',
       '2021-24', '2021-25', '2021-26', '2021-27'], dtype=object)

La dernière semaine concernée est vers le milieu d'année, et c'est bien en été que la campagne de vaccination a pris une accélération sérieuse... Donc ? Donc **on va remplacer les cellules NaN par des 0**, ce qui va rendre possibles les conversions des valeurs concernées en entiers :

In [37]:
compavac=compavac.fillna(0)
compavac["effectif_cumu_termine"]=compavac["effectif_cumu_termine"].astype(int)
compavac["population_carto"]=compavac["population_carto"].astype(int)
compavac["taux_cumu_termine"]=compavac["taux_cumu_termine"]*100
compavac.head()

,date_reference,date,semaine_injection,epci,libelle_epci,libelle_classe_age,population_carto,effectif_cumu_termine,taux_cumu_termine
0,2021-12-26,2021-02-14,2021-06,200000545,CC DES PORTES DE ROMILLY SUR SEINE,moins de 20 ans,4230,0,0.0
1,2021-12-26,2021-02-14,2021-06,200000545,CC DES PORTES DE ROMILLY SUR SEINE,de 40 à 54 ans,3200,40,1.2
2,2021-12-26,2021-02-14,2021-06,200000545,CC DES PORTES DE ROMILLY SUR SEINE,de 20 à 39 ans,3760,10,0.3
3,2021-12-26,2021-02-14,2021-06,200000545,CC DES PORTES DE ROMILLY SUR SEINE,de 65 à 74 ans,2350,20,0.9
4,2021-12-26,2021-02-14,2021-06,200000545,CC DES PORTES DE ROMILLY SUR SEINE,75 ans et plus,2190,40,1.7


Nous avons des données à l'échelle des EPCI, comment passer à l'échelle régionale ? On peut très facilement regrouper par colonne [grâce à la fonction groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) de pandas, qui inclut des opérations de regroupement.

On va, dans l'ordre :
* **regrouper la DF par semaine d'injection et classe d'âge**, en additionnant les valeurs de chaque catégorie de groupe (l'ensemble des moins de 20 ans de la semaine 2021-06, des 20-39 ans de la même semaine, etc...)
* ne sélectionner que les colonnes intéressantes
* refaire quelques conversions en nombres entiers
* **recalculer le taux de vaccinations complètes** à partir des sommes calculées précédemment :

In [38]:
tampon_vac=compavac.groupby(["semaine_injection","libelle_classe_age"]).sum().reset_index()
tampon_vac=tampon_vac[["semaine_injection","libelle_classe_age","population_carto","effectif_cumu_termine"]]
tampon_vac["effectif_cumu_termine"]=tampon_vac["effectif_cumu_termine"].astype(int)
tampon_vac["population_carto"]=tampon_vac["population_carto"].astype(int)
tampon_vac["taux_cumu_termine"]=round((tampon_vac["effectif_cumu_termine"]/tampon_vac["population_carto"])*100,2)
tampon_vac

,semaine_injection,libelle_classe_age,population_carto,effectif_cumu_termine,taux_cumu_termine
0,2021-06,75 ans et plus,569770,25900,4.55
1,2021-06,de 20 à 39 ans,1233820,4070,0.33
2,2021-06,de 40 à 54 ans,1028520,12950,1.26
3,2021-06,de 55 à 64 ans,722320,16020,2.22
4,2021-06,de 65 à 74 ans,643440,6220,0.97
...,...,...,...,...,...
271,2021-51,de 20 à 39 ans,1233820,956050,77.49
272,2021-51,de 40 à 54 ans,1028520,865520,84.15
273,2021-51,de 55 à 64 ans,722320,636290,88.09
274,2021-51,de 65 à 74 ans,643440,579120,90.00


Tout cela est bel et bon, mais il n'y a encore des semaines d'injection qui se répètent : chacune est dupliquée six fois (une fois par tranche d'âge), or il serait bien plus pratique de n'avoir qu'une ligne par semaine d'injection dans notre base de données.

C'est le moment **de passer [en mode tableau croisé dynamique](https://openclassrooms.com/fr/courses/7139456-perfectionnez-vous-sur-excel?archived-source=825502)** en visant la colonne libelle_classe_age. 

Pour le dire vite, chaque colonne de valeurs ciblée (les effectifs et taux de vaccinations complètes, dans notre cas) vont être reformatée en autant de colonnes qu'il y a de valeurs uniques dans libelle_classe_age. On considère en index la semaine d'injection et on devrait s'en sortir sans encombre.

Démonstration :

In [39]:
test=tampon_vac.pivot_table(index=['semaine_injection'], columns=['libelle_classe_age'], values=["effectif_cumu_termine","taux_cumu_termine"])
test.columns=test.columns.map('_'.join)
test=test.reset_index()
test.head()

,semaine_injection,effectif_cumu_termine_75 ans et plus,effectif_cumu_termine_de 20 à 39 ans,effectif_cumu_termine_de 40 à 54 ans,effectif_cumu_termine_de 55 à 64 ans,effectif_cumu_termine_de 65 à 74 ans,effectif_cumu_termine_moins de 20 ans,taux_cumu_termine_75 ans et plus,taux_cumu_termine_de 20 à 39 ans,taux_cumu_termine_de 40 à 54 ans,taux_cumu_termine_de 55 à 64 ans,taux_cumu_termine_de 65 à 74 ans,taux_cumu_termine_moins de 20 ans
0,2021-06,25900,4070,12950,16020,6220,0,4.55,0.33,1.26,2.22,0.97,0.0
1,2021-07,54610,6310,17630,21170,9970,0,9.58,0.51,1.71,2.93,1.55,0.0
2,2021-08,79130,8380,21140,24690,13370,0,13.89,0.68,2.06,3.42,2.08,0.0
3,2021-09,100220,9540,23430,27440,16620,10,17.59,0.77,2.28,3.80,2.58,0.0
4,2021-10,119960,10820,25520,30020,20690,20,21.05,0.88,2.48,4.16,3.22,0.0


Six valeurs uniques dans la colonne deux références, deux colonnes de valeurs, une colonne d'index... Cela donne bien 1+6\*2 colonnes à l'arrivée.

On va aller encore plus loin en regroupant dans un dictionnaire Python les effectifs et taux de vaccinations complètes par classe d'âge, sur le modèle *{"moins de 20 ans":{"effectif_cumu_termine":n1,"taux_cumu_termine":m1},"20-39 ans":{"effectif_cumu_termine":n2,"taux_cumu_termine":m2}}*

Pour se faire, on va considérer chaque ligne d'une DF [grâce à la fonction iterrows()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html) de pandas. Gros avantage de cette fonction : la ligne s'interroge comme un dictionnaire et on peut donc créer le résultat attendu avec [une petite compréhension de dictionnaires des familles](https://www.python.org/dev/peps/pep-0274/) :

In [40]:
cols_ref=["effectif_cumu_termine","taux_cumu_termine"]
# on va faire les choses bien, et tout ranger dans une liste
liste_synthese=[]
for i, ligne in test.iterrows():
    # à chaque ligne, on ajoute à la liste un dico
    # chaque clé du dico est une valeur tiré d'une liste
    # à chaque clé est associé un dictionnaire colonne de référence/valeur associée d'une DF
    liste_synthese.append({a:{c:ligne[c+"_"+a] for c in cols_ref} for a in list(tampon_vac["libelle_classe_age"].unique())})
print(liste_synthese[0])

{'75 ans et plus': {'effectif_cumu_termine': 25900, 'taux_cumu_termine': 4.55}, 'de 20 à 39 ans': {'effectif_cumu_termine': 4070, 'taux_cumu_termine': 0.33}, 'de 40 à 54 ans': {'effectif_cumu_termine': 12950, 'taux_cumu_termine': 1.26}, 'de 55 à 64 ans': {'effectif_cumu_termine': 16020, 'taux_cumu_termine': 2.22}, 'de 65 à 74 ans': {'effectif_cumu_termine': 6220, 'taux_cumu_termine': 0.97}, 'moins de 20 ans': {'effectif_cumu_termine': 0, 'taux_cumu_termine': 0.0}}


Ca a l'air d'avoir marché, et on va même aller encore plus loin en factorisant ceci dans une fonction :

In [41]:
def synthese_col(DF:pd.core.frame.DataFrame,col_valeurs:list,titres_valeurs:list):
    """
        Cette liste reformate des valeurs dispersées sur plusieurs colonnes d'une DF dans une liste de dictionnaires.
        Chaque dictionnaire a en clés différentes en-tête auxquelles on attache des valeurs distinctes.
        Par exemple : rassembler les taux et effectifs de vaccination (valeurs distinctes) par classes d'âge (en-tête)
    """
    liste_synthese=[]
    for i, ligne in DF.iterrows():
        liste_synthese.append({t_v:{col_v:ligne[col_v+"_"+t_v] for col_v in col_valeurs} for t_v in titres_valeurs})
    return liste_synthese

Et vérifier par un petit test, pour la route :

In [42]:
synthese_col(test,["effectif_cumu_termine","taux_cumu_termine"],list(tampon_vac["libelle_classe_age"].unique()))[0]

{'75 ans et plus': {'effectif_cumu_termine': 25900, 'taux_cumu_termine': 4.55},
 'de 20 à 39 ans': {'effectif_cumu_termine': 4070, 'taux_cumu_termine': 0.33},
 'de 40 à 54 ans': {'effectif_cumu_termine': 12950, 'taux_cumu_termine': 1.26},
 'de 55 à 64 ans': {'effectif_cumu_termine': 16020, 'taux_cumu_termine': 2.22},
 'de 65 à 74 ans': {'effectif_cumu_termine': 6220, 'taux_cumu_termine': 0.97},
 'moins de 20 ans': {'effectif_cumu_termine': 0, 'taux_cumu_termine': 0.0}}

Cela a bien marché, on va donc créer une nouvelle DF avec les données du Grand-Est :

In [43]:
def_vac=test.copy()
def_vac["Grand-Est"]=synthese_col(def_vac,["effectif_cumu_termine","taux_cumu_termine"],list(tampon_vac["libelle_classe_age"].unique()))
def_vac=def_vac[["semaine_injection","Grand-Est"]]
def_vac.head()

,semaine_injection,Grand-Est
0,2021-06,{'75 ans et plus': {'effectif_cumu_termine': 2...
1,2021-07,{'75 ans et plus': {'effectif_cumu_termine': 5...
2,2021-08,{'75 ans et plus': {'effectif_cumu_termine': 7...
3,2021-09,{'75 ans et plus': {'effectif_cumu_termine': 1...
4,2021-10,{'75 ans et plus': {'effectif_cumu_termine': 1...


Et maintenant ? On va aller encore plus loin en factorisant les manips de départ et en appelant synthese_col **en fin de nouvelle fonction** :

In [44]:
def synthese_pivot(DF:pd.core.frame.DataFrame,selec:list,l_index:list,l_col:list,l_val:list):
    """
        Cette fonction fait pivoter une BDD à partir de données de références qui seront ensuite rassemblées en clés de dictionnaire.
        Par exemple : rassembler les données quotidiennes de vaccination de classes d'âge par jour.
    """
    DF_trans=DF[DF[selec[0]]==selec[1]]
    unique_col=list(DF_trans[l_col[0]].unique())
    tampon=DF_trans.pivot_table(index=l_index, columns=l_col, values=l_val)
    tampon.columns=tampon.columns.map('_'.join)
    tampon=tampon.reset_index()
    return synthese_col(tampon,l_val,unique_col)

Il ne reste plus qu'à créer à la volée deux nouvelles colonnes centrées sur Mulhouse et le Kochersberg :

In [45]:
# pour rappel, zip permet d'itérer en parallèle des variables qui ont la même taille
# ici t_col = "Kochersberg" / col="CC DU KOCHERSBERG" à l'indice 0, t_col = "Mulhouse" / col="CA MULHOUSE ALSACE AGGLOMERATION" à l'indice 1
for t_col, col in zip(["Kochersberg","Mulhouse"],["CC DU KOCHERSBERG","CA MULHOUSE ALSACE AGGLOMERATION"]):
    def_vac[t_col]=synthese_pivot(compavac,["libelle_epci",col],["semaine_injection"],["libelle_classe_age"],["effectif_cumu_termine","taux_cumu_termine"])
def_vac.head()

,semaine_injection,Grand-Est,Kochersberg,Mulhouse
0,2021-06,{'75 ans et plus': {'effectif_cumu_termine': 2...,{'de 20 à 39 ans': {'effectif_cumu_termine': 2...,{'moins de 20 ans': {'effectif_cumu_termine': ...
1,2021-07,{'75 ans et plus': {'effectif_cumu_termine': 5...,{'de 20 à 39 ans': {'effectif_cumu_termine': 3...,{'moins de 20 ans': {'effectif_cumu_termine': ...
2,2021-08,{'75 ans et plus': {'effectif_cumu_termine': 7...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...
3,2021-09,{'75 ans et plus': {'effectif_cumu_termine': 1...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...
4,2021-10,{'75 ans et plus': {'effectif_cumu_termine': 1...,{'de 20 à 39 ans': {'effectif_cumu_termine': 6...,{'moins de 20 ans': {'effectif_cumu_termine': ...


Dernière coquetterie, on va remplacer les semaines d'injection par les lundis de ces semaines, plus faciles pour se repérer dans le temps.

Une solution peut-être de créer une liste de dates, qu'on remplit comme ceci à partir d'un départ fixé :

In [46]:
l_dates=[]
depart=datetime.datetime.strptime("08/02/2021", "%d/%m/%Y")
for i in range(len(def_vac)):
    l_dates.append((depart+datetime.timedelta(days=i*7)).strftime("%d/%m/%Y"))
l_dates[:5]

['08/02/2021', '15/02/2021', '22/02/2021', '01/03/2021', '08/03/2021']

Cette liste ayant la même taille que def_vac, on peut l'utiliser pour remplir une nouvelle colonne :

In [47]:
def_vac["date"]=l_dates
def_vac.tail()

,semaine_injection,Grand-Est,Kochersberg,Mulhouse,date
41,2021-47,{'75 ans et plus': {'effectif_cumu_termine': 4...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...,22/11/2021
42,2021-48,{'75 ans et plus': {'effectif_cumu_termine': 4...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...,29/11/2021
43,2021-49,{'75 ans et plus': {'effectif_cumu_termine': 4...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...,06/12/2021
44,2021-50,{'75 ans et plus': {'effectif_cumu_termine': 4...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...,13/12/2021
45,2021-51,{'75 ans et plus': {'effectif_cumu_termine': 4...,{'de 20 à 39 ans': {'effectif_cumu_termine': 5...,{'moins de 20 ans': {'effectif_cumu_termine': ...,20/12/2021


Plus qu'à exclure la colonne semaine_injection et à enregistrer le json :

In [49]:
def_vac=def_vac[["date","Mulhouse","Grand-Est","Kochersberg"]]
def_vac.to_json("data/alvac.json",orient="records")